In [1]:
import gpytorch
import numpy as np
import pandas as pd
import torch

from bo_functions_GP_Analysis import LSO_LOO_Analysis
from datetime import datetime

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
#--------------------------------

#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = None ##For Testing

#Set Parameters
# emulator = True
emulator = np.array([True, False])
# obj = "obj"
obj = np.array(["obj", "LN_obj"])
save_fig = True
verbose = True

LOO = False
LSO = True

Theta_True = np.array([1,-1])
train_iter = 300
noise_std = 0.1
shuffle_seed = 9
set_lengthscale = None

#Pull Experimental data from CSV
exp_data_doc = 'Input_CSVs/Exp_Data/n=5.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1]
Yexp = exp_data[:,2]
n = len(Xexp)

#Define GP Testing space
p=20
Theta1 =  np.linspace(-2,2,p) #1x10
Theta2 =  np.linspace(-2,2,p) #1x10
theta_mesh = np.array(np.meshgrid(Theta1, Theta2)) #2 Uniform 5x5 arrays

for emul in emulator:
    
    print("Emulator?:", emul)
    print("-------------------")
    if emul == True:
        len_data = 100
        obj_use = np.array(["obj"])
    else:
        obj_use = obj
        len_data = 20
    for obj_func in obj_use:
        print("Objective Function:", obj_func)
        print("Lengthscale Set To:", set_lengthscale)
        results = LSO_LOO_Analysis(theta_mesh,Theta_True,train_iter, Xexp, Yexp,
                                         noise_std, obj_func, emul, set_lengthscale, len_data, verbose, 
                                         save_fig, shuffle_seed, DateTime, LOO, LSO)
        print("-  -  -  -  -  -  -  -  -  -  -")

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Date and Time:  12-Sep-2022 (15:09:55)
Emulator?: True
-------------------
Objective Function: obj
Lengthscale Set To: None
Train/Test Separation Factor 0.95


/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:130: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution


Train/Test Separation Factor 0.5
-  -  -  -  -  -  -  -  -  -  -
Emulator?: False
-------------------
Objective Function: obj
Lengthscale Set To: None
Train/Test Separation Factor 0.95
Train/Test Separation Factor 0.5


/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/numpy/core/shape_base.py:65: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  ary = asanyarray(ary)
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/numpy/core/shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary

-  -  -  -  -  -  -  -  -  -  -
Objective Function: LN_obj
Lengthscale Set To: None
Train/Test Separation Factor 0.95
Train/Test Separation Factor 0.5


/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/numpy/core/shape_base.py:65: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  ary = asanyarray(ary)
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/numpy/core/shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary

-  -  -  -  -  -  -  -  -  -  -
